AI computer assignment report

Kasra Ghorbani

810101489

Sources : http://www.sc.ehu.es/ccwbayes/docencia/kzmm/files/AG-knapsack.pdf

https://www.youtube.com/watch?v=MacVqujSXWE

Topic : Using genetic algorithms to find a valid solution to the "Knapsack problem"

The Knapsack problem : Say we want to go on a picnic and we have a knapsack with limited storage available to us. find a way to fill the bag while 1 - the weight of the items not exceeding the maximum allowed weight 2 - the value of the items carried being above a minimum value

To solve this problem , as mentioned above we use genetic algorithms. The process can be broken down to: 
1 - Creating a number of random solutions called chromosomes and creating a population (group) of solutions
2 - Assigning a fitness score to each solutions using a fitness function. The function should give meaningfull scores to each solution in order for us to be able to reach a decent solution through crossovers and mutations.
3 - Creation of a new generation of chromosomes to replace the old population by means of "crossovers" and "mutations"

- Crossover : The act of taking two chromosomes randomly chosen by their fitness scores and mixing their attributes, creating new chromosomes that carry attributes of both of their parents. We used single-point crossovers for this algorithm

- Mutations : The act of changing the genes of a chromosome randomly with the use of a probabilty called the "mutation rate". Thus diversifying our population to avoid reaching a local non-optimal solution for the problem

- Elitism : Avoid losing the best chromosomes of each generation by passing them on onto the next generation.

4 - Putting together all of the previously mentioned components to implement the genetic algorithm. We will explain each part of the code when needed

5 - Reaching a solution


In [ ]:
max_weight = int(input())
min_value = int(input())
allowed_num_of_snacks_str = input().split(' ')
min_num_of_snacks = int(allowed_num_of_snacks_str[0])
max_num_of_snacks = int(allowed_num_of_snacks_str[1])
num_of_generations = 50
pop_size = 1500
elites_size = 0.1 * pop_size
mutation_rate = 0.05
df = pd.read_csv('snacks.csv')

From the above code we will only explain :
pop_size : is the population size. We chose a relatively common number here. 1500 is not too large so we can get an answer in a decent time and it is large enough to make sure we have enough chromosomes. Smaller numbers also worked but did not always produce the best results.
elites_size : We want the top 10% of our population to ripple through the next generation without losing them.
mutation_rate : the chance of a gene undergoing a mutation. should usually be a low number such as 0.01 or 0.05. makes sure that our population maintains its diversity.
 

In [ ]:
def initialize_population(pop_size, min_num_of_snacks, max_num_of_snacks, df):
    population = []
    for i in range(0, pop_size):
        chromosome = [0] * len(df)
        num_of_items = random.randint(min_num_of_snacks, max_num_of_snacks)
        chosen_items_index = random.sample(range(len(df)), num_of_items)
        for item_index in chosen_items_index:
            weight_of_each_item = random.uniform(
                0, df.loc[item_index]['Available Weight'])
            chromosome[item_index] = weight_of_each_item
        population.append(chromosome)
    return population

The above code is a function that creates our beginning population. we create the amount of chromosomes required by choosing a random but ALLOWED number of snacks for the knapsack solution. while this isn't completely random it greatly improves our performance and the time it takes to find a good answer. I wasn't only going for a plain answer but rather a complete answer through evolving already good solutions into better ones.
We also choose a valid weight of each item further improving our chromosomes.

In [ ]:
def calculate_fitnesses(population, max_weight, min_value, df):
    fitnesses = []
    total_fitness = 0
    for chromosome in population:
        fitness = calc_fitness(chromosome, max_weight, min_value, df)
        fitnesses.append(fitness)
        total_fitness += fitness
    return fitnesses, total_fitness


def calc_fitness(chromosome, max_weight, min_value, df):
    total_weight = 0
    total_value = 0
    for item_index, item_weight in enumerate(chromosome):
        total_weight += item_weight
        total_value += (item_weight / df.loc[item_index]
                        ['Available Weight']) * df.loc[item_index]['Value']
    if total_weight > max_weight or total_value < min_value:
        fitness = 0.1
    else:
        fitness = total_value
    return fitness

The above functions calculate the fitness of each chromosome in the population. This will be used later on to calculate the mating chance of each chromosome which is the chromosome's chance of performing a crossover with other chromosomes. the fitness of each chromosome is for the most part determined by the total value it carries since the weight and the number of problems have already been taken care of. however we still consider other solutions to have a small fitness score so they're not entirely wiped out from the population.

In [ ]:
def mutate(chromosome, mutation_rate, df):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.uniform(0, df.loc[i]['Available Weight'])
    return chromosome


def create_new_population(population, fitnesses, pop_size, elites_size, mating_chance):
    new_population = []
    population_np = np.array(population)
    fitnesses_np = np.array(fitnesses)
    sorted_indices = np.argsort(fitnesses_np)[::-1]
    sorted_population = population_np[sorted_indices]
    elite_population = sorted_population[:int(elites_size)].tolist()
    new_population.extend(elite_population)

    for _ in range(0, (pop_size // 2) - len(elite_population)):
        chosen_chromosomes = random.choices(
            population, weights=mating_chance, k=2)
        child1 = chosen_chromosomes[0][:len(
            df)//2] + chosen_chromosomes[1][len(df)//2:]
        child2 = chosen_chromosomes[1][:len(
            df)//2] + chosen_chromosomes[0][len(df)//2:]
        child1 = mutate(child1, mutation_rate, df)
        child2 = mutate(child2, mutation_rate, df)
        new_population.append(child1)
        new_population.append(child2)
    return new_population

The create_new_population function does the following to create a new population to replace the old one :

1 - Sort the current population with respect to their fitness in descending order and choose the elite chromosomes

2 - move the elite chromosomes onto the next generation

3 - Randomly choose 2 chromosomes based on their fitness scores and perform crossovers on them.

4 - mutate the children and add them into the new population

the mutation procedure consists of choosing a gene to mutate based on a small number called the mutation rate and how it works is we replace the current weight of the gene with a random but still valid weight. This will help keep the population diverse and keep it from losing all diversity where all chromosomes are similar.

Through some testing it is safe to say that this algorithm works for most of the inputs provided.
Answer to the questions :
1 - a vast population makes the algorithm slow and also might lead to having similar chromosomes if the space isn't very big. On the other hand a small population might cause the problem of not having enough answers with a good fitness or no answers at all!
So it's important to choose a reasonable number as our population size.

2 - If the population grows with each new generation it might lead to :
* : Increased computational demand
** : "Potential" for better solutions as our population has more chromosomes to choose from
*** : Having more chromosomes in our population makes the convergence slower meaning that it takes more and more time for our answers to mature into good solutions.

3 - Crossover happens between 2 different chromosomes and is meant to create better chromosomes. On the other hand mutation happens on one chromosome and only alters some of it's genes by a small chance. We use crossovers to improve the fitness of our population but use mutations to diversify our population so we don't reach a "local optimal" solution. kinda similar to simulated annealing.

4 - The stop in the evolution of chromosomes might be sign that our population wasn't diverse enough so all solutions are mostly the same and don't contribute much. Also the selection pressure might be too high making it difficult for most chromosomes to survive and therefore losing diversity. A low mutation rate might cause this as well.

5 - We can stop the program after the maximum number of generations have been reached. Another way can be checking if a feasible solution exists at all using mathematics for example. 